<a href="https://colab.research.google.com/github/SutirthaChakraborty/AI-Dance-based-on-Human-Pose-Estimation/blob/main/AUDIO_TO_MOVES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !gdown --id 1ElfggfgCenCXNFNRpOgBQe2YPC8E9Sqj
# !unzip /content/Midi_Voilin_channel.zip
# https://drive.google.com/file/d/1ElfggfgCenCXNFNRpOgBQe2YPC8E9Sqj/view?usp=sharing

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# **GET FEATURES**

In [1]:
!pip install mediapipe

     |████████████████████████████████| 32.7 MB 1.3 MB/s 


In [2]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 

from scipy.signal import savgol_filter
def my_filter(x):
    return savgol_filter(x, 11, 1)



# **Generate the dataset**

In [ ]:
def extract_keypoints(results):
    # pose = np.array([[res.x, res.y, res.z] for res in results.pose_world_landmarks.landmark]).flatten() if results.pose_world_landmarks else np.zeros(33*3)
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [ ]:
# import glob
# listofFile= (glob.glob("/content/Midi_Voilin_channel1/*.mp4")) 


# fullData=np.empty([0,225+20+1])
# # fullData=np.empty([0,99+20])

# import librosa
# import librosa.display
# import matplotlib.pyplot as plt
# import warnings
# warnings.filterwarnings("ignore", category=Warning)

# from tqdm import tqdm_notebook as tqdm
# k=1

# for i in tqdm(listofFile):     # -------------------------------------> change the file numbers. remove 1 for all files
#   filename =i
#   y, sr = librosa.load(filename)
#   cap = cv2.VideoCapture(filename)
#   fps = cap.get(cv2.CAP_PROP_FPS)
#   timestamps = [cap.get(cv2.CAP_PROP_POS_MSEC)]
#   calc_timestamps = [0.0]
#   # Initiate holistic model
#   totalframecount= int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#   i = 1
#   pbar = tqdm(total = int(totalframecount))


#   Posedata=[]
#   count=0
#   with mp_holistic.Holistic(
#     static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as holistic:
#     try:
#       while cap.isOpened():
#         pbar.update(i)
#         ret, frame = cap.read()
#         # Recolor Feed
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         results = holistic.process(image)
#         a=extract_keypoints(results)
#         Posedata.append(a)
#         timestamps.append(cap.get(cv2.CAP_PROP_POS_MSEC)/1000)

  
#     except Exception as e: 
#       print(e,i)
#       pass

#   alldata_raw=pd.DataFrame(np.array(Posedata))
#   Posedata = np.array(alldata_raw.apply(my_filter))

#   # Sound Features
#   Posedata=np.array(Posedata)
#   timestamps=np.array(timestamps[:-1])
#   mfccs= librosa.feature.mfcc(y=y, sr=sr).T[librosa.time_to_frames(timestamps)]
#   S = np.abs(librosa.stft(y))
#   db_median = librosa.amplitude_to_db(S, ref=np.median)[0][librosa.time_to_frames(timestamps)]
#   new_lst = [[i] for i in db_median] 
#   data= np.concatenate([np.array(new_lst),mfccs,Posedata], axis=1)
#   filename= str(k)+".npy"
#   k+=1
#   # with open(filename, 'wb') as f:
#   #   np.save(f,data)

#   fullData= np.concatenate([fullData,data], axis=0)
#   print(fullData.shape)

# X=fullData[:,:-225]     # INPUTS
# y=fullData[:,-225:]     # OUTPUT

**Save the dataset**

In [ ]:
# filename="/content/drive/MyDrive/MusicGeneration/PoseNumpyBlender/RealWorldFull.npy"
# filename= "posedata.npy"
# with open(filename, 'wb') as f:
#   np.save(f,fullData)

# from google.colab import files
# files.download(filename) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Download the dataset from googleDrive and run**

In [3]:
# !gdown --id 11-MZZ-d8VUMGu0kJ1ioq9QGiKRf3UoDQ
!gdown --id 1VquzQcKhUy79BFdYK_MCCFOaQv1EBpWY
import numpy as np
data = np.load('/content/posedataFull.npy')
X=data[:,:-225]     # INPUTS
y=data[:,-225:]     # OUTPUT
X.shape,y.shape
 

Downloading...
From: https://drive.google.com/uc?id=1VquzQcKhUy79BFdYK_MCCFOaQv1EBpWY
To: /content/posedataFull.npy
100% 134M/134M [00:01<00:00, 79.7MB/s]


((68101, 21), (68101, 225))

# **ONLY FOR CONDUCTOR** (OTHERWISE DONT RUN)

In [ ]:
X=np.flipud(X)
y=np.flipud(y)

# **Applying to Model**

In [4]:
def preprocess(X, Y):
    test_data = X.shape[0] - int(X.shape[0]/20) # 5% data is used for predictions
    X_train = X[:test_data, :]
    Y_train = Y[:test_data, :]
    X_test = X[test_data:, :]
    Y_test = Y[test_data:, :]
    
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    Y_train = scaler.fit_transform(Y_train)
    Y_test = scaler.transform(Y_test)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
    
    return X_train, X_test, Y_train, Y_test, scaler

**DL IMPORTS**

In [5]:
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM ,Bidirectional
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf 
import cv2
import time
import numpy as np
import pandas as pd
import librosa
from keras.models import Sequential
from keras.layers import Dense, LSTM,Dropout
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from pickle import load
from pickle import dump

**SCALE and SPLIT**

**GENERATE THE MODEL**

In [ ]:
X_train, X_test, Y_train, Y_test, scaler = preprocess(X, y)
# dump(scaler, open('/content/drive/MyDrive/MusicGeneration/PoseNumpyBlender/scaler.pkl', 'wb'))
# scaler = load(open('/content/drive/MyDrive/MusicGeneration/PoseNumpyBlender/scaler.pkl', 'rb'))
in_dim = (X_test.shape[1], X_test.shape[2])
out_dim = y.shape[1]

print(in_dim)

keras.backend.clear_session()
model = Sequential()
model.add(LSTM(units = 128, input_shape = in_dim, return_sequences = True))
model.add(Bidirectional(LSTM(54, activation = 'tanh')))
model.add(Dense(out_dim))

model.compile(optimizer = 'adam', loss = 'mean_squared_error')


checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_best_only=True)

history = model.fit(X_train, Y_train, batch_size = 16, epochs = 600,callbacks=[model_checkpoint_callback])

dump(model, open('model.pkl', 'wb'))
# dump(model, open('/content/drive/MyDrive/MusicGeneration/PoseNumpyBlender/model.pkl', 'wb'))

(21, 1)
Epoch 1/600
4044/4044 [==============================] - 30s 6ms/step - loss: 0.0196
Epoch 2/600
4044/4044 [==============================] - 24s 6ms/step - loss: 0.0171
Epoch 3/600
4044/4044 [==============================] - 25s 6ms/step - loss: 0.0150
Epoch 4/600
4044/4044 [==============================] - 24s 6ms/step - loss: 0.0134
Epoch 5/600
4044/4044 [==============================] - 24s 6ms/step - loss: 0.0125
Epoch 6/600
4044/4044 [==============================] - 23s 6ms/step - loss: 0.0118
Epoch 7/600
4044/4044 [==============================] - 23s 6ms/step - loss: 0.0112
Epoch 8/600
4044/4044 [==============================] - 23s 6ms/step - loss: 0.0107
Epoch 9/600
4044/4044 [==============================] - 23s 6ms/step - loss: 0.0103
Epoch 10/600
4044/4044 [==============================] - 23s 6ms/step - loss: 0.0100
Epoch 11/600
4044/4044 [==============================] - 23s 6ms/step - loss: 0.0097
Epoch 12/600
4044/4044 [==============================]

In [ ]:
!pip install plot-keras-history

In [ ]:
from plot_keras_history import show_history, plot_history
import matplotlib.pyplot as plt
show_history(history)
plot_history(history, path="standard.png")
plt.close()

In [ ]:
predictions = scaler.inverse_transform(model.predict(X_test)) # Split data, train lstm and return predictions.

In [ ]:
X_test.shape,predictions.shape

In [ ]:
  alldata_raw=pd.DataFrame(np.array(predictions))
  PosedataPrediction = np.array(alldata_raw.apply(my_filter))
  print(PosedataPrediction.shape)

In [ ]:
np.save("output.npy", PosedataPrediction)

In [ ]:
from google.colab import files
files.download("output.npy") 

# **TESTING**

In [ ]:
import cv2
cam = cv2.VideoCapture("/content/Midi_Voilin_channel1/v17.mp4")
fps = cam.get(cv2.CAP_PROP_FPS)
fps

In [ ]:
import glob
 
import librosa
import librosa.display
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=Warning)

from tqdm import tqdm_notebook as tqdm

fullData=np.empty([0,225+20+1])

import librosa
import librosa.display
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import MinMaxScaler


warnings.filterwarnings("ignore", category=Warning)

from pickle import load
# load the model
model = load(open('/content/drive/MyDrive/MusicGeneration/PoseNumpyBlender/model.pkl', 'rb'))
# load the scaler
scaler = load(open('/content/drive/MyDrive/MusicGeneration/PoseNumpyBlender/scaler.pkl', 'rb'))


k=1

filename ="/content/Midi_Voilin_channel1/v17.mp4"
y, sr = librosa.load(filename)
cap = cv2.VideoCapture(filename)
fps = cap.get(cv2.CAP_PROP_FPS)
timestamps = [cap.get(cv2.CAP_PROP_POS_MSEC)]
calc_timestamps = [0.0]
# Initiate holistic model
totalframecount= int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
i = 1
pbar = tqdm(total = int(totalframecount))


Posedata=[]
count=0
with mp_holistic.Holistic( static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as holistic:
  try:
    while cap.isOpened():
      pbar.update(i)
      ret, frame = cap.read()
      # Recolor Feed
      image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
      results = holistic.process(image)
      a=extract_keypoints(results)
      Posedata.append(a)
      timestamps.append(cap.get(cv2.CAP_PROP_POS_MSEC)/1000)


  except Exception as e: 
    print(e,i)
    pass

alldata_raw=pd.DataFrame(np.array(Posedata))
Posedata = np.array(alldata_raw.apply(my_filter))

# Sound Features
Posedata=np.array(Posedata)
timestamps=np.array(timestamps[:-1])
mfccs= librosa.feature.mfcc(y=y, sr=sr).T[librosa.time_to_frames(timestamps)]
S = np.abs(librosa.stft(y))
db_median = librosa.amplitude_to_db(S, ref=np.median)[0][librosa.time_to_frames(timestamps)]
new_lst = [[i] for i in db_median] 
data= np.concatenate([np.array(new_lst),mfccs,Posedata], axis=1)
fullData= np.concatenate([fullData,data], axis=0)


X_testing=fullData[:,:-225]     # INPUTS
Y_testing=fullData[:,-225:]     # OUTPUT

scaler = MinMaxScaler()
X_testing = scaler.fit_transform(X_testing)
Y_testing = scaler.fit_transform(Y_testing)
X_testing = X_testing.reshape(X_testing.shape[0], X_testing.shape[1], 1)
predictions = scaler.inverse_transform(model.predict(X_testing)) # Split data, train lstm and return predictions.
alldata_raw=pd.DataFrame(np.array(predictions))
PosedataPrediction = np.array(alldata_raw.apply(my_filter))
print(PosedataPrediction.shape)
np.save("output.npy", PosedataPrediction)

# **BLENDER CODE**

In [ ]:

# import cv2
# import mediapipe as mp
# import bpy
# import sys
# from mathutils import Vector
# import math
# from scipy.signal import savgol_filter
# import pandas as pd
# import numpy as np


# alldata_smoothed = np.load("sample.npy")




# def my_filter(x):
#     return savgol_filter(x, 11, 1)


# def middle_point(p1,p2,p_middle):
#     bpy.ops.object.select_all(action='DESELECT')
#     bpy.data.objects[p1].select_set(True)
#     bpy.data.objects[p2].select_set(True)
#     bpy.context.view_layer.objects.active = bpy.data.objects[p2]
#     obs = bpy.context.selected_objects
#     n = len(obs)
# #    print('n: ',n)
#     assert(n)
#     #scene.cursor.location = sum([o.matrix_world.translation for o in obs], Vector()) / n
#     bpy.data.objects[p_middle].location = sum([o.matrix_world.translation for o in obs], Vector()) / n



# def get_landmarks(vid_name, frame_list):
#     mp_drawing = mp.solutions.drawing_utils
#     mp_pose = mp.solutions.pose
#     full=[]
#     global alldata_smoothed
    
#     # For static images:
# #    pose = mp_pose.Pose(static_image_mode=True, model_complexity=2)
# #    for idx, image in enumerate(frame_list):
# #        image_height, image_width, _ = image.shape
#         # Convert the BGR image to RGB before processing.
# #        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

# #        poses = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(99)
# #        full.append(poses)
        
    
# #    alldata_raw=pd.DataFrame(np.array(full))
# #    alldata_smoothed = np.array(alldata_raw.apply(my_filter)) 
    
# #    frame_list=alldata_smoothed.shape[0]
    
#     for idx, image in enumerate(frame_list):
#         x=0
#         y=1
#         z=2
#         scale = 2
#         print('frame: ',idx)
#         j=alldata_smoothed[idx]
#         try:
# #            len(results.pose_landmarks.landmark)
#             t=0
#             for i in range(75):
#                 if i == 33:
#                     t=36
#                     print(j.shape)
#                 x_pose = (0.5-j[i*3])*scale
#                 y_pose = (0.5-j[i*3+1])*scale
#                 z_pose = j[i*3+2]*0.5
                
                
#                 bpy.data.objects["Point."+str(1000+t)[1:]].location[x]=x_pose
#                 bpy.data.objects["Point."+str(1000+t)[1:]].location[y]=y_pose
#                 bpy.data.objects["Point."+str(1000+t)[1:]].location[z]=z_pose
#                 if t == 10:
#                     middle_point('Point.009','Point.010','Point.033')
#                     bpy.data.objects["Point."+str(1000+33)[1:]].keyframe_insert(data_path="location", frame=idx)
#                 if t == 12:
#                     middle_point('Point.011','Point.012','Point.034')
#                     bpy.data.objects["Point."+str(1000+34)[1:]].keyframe_insert(data_path="location", frame=idx)
#                 if t == 24:
#                     middle_point('Point.023','Point.024','Point.035')
#                     bpy.data.objects["Point."+str(1000+35)[1:]].keyframe_insert(data_path="location", frame=idx)
                    
#                 #---------------------------------for extra points of hand-----------------------------------
#                 if t == 45: #Left
#                     middle_point('Point.036','Point.045','Point.078')
#                     bpy.data.objects["Point."+str(1000+78)[1:]].keyframe_insert(data_path="location", frame=idx)
#                 if t == 66: #Right
#                     middle_point('Point.057','Point.066','Point.079')
#                     bpy.data.objects["Point."+str(1000+79)[1:]].keyframe_insert(data_path="location", frame=idx)
                
                            
#                 bpy.data.objects["Point."+str(1000+t)[1:]].keyframe_insert(data_path="location", frame=idx)
#                 t+=1
            
#     #
# #                print('frame: ',idx,' landmark_id: ',i,'x: ', x_pose, ' - y: ',y_pose,' - z: ',z_pose)
#         except:
#             print('Error Frame: ',idx)
#             bpy.data.objects["Point."+str(1000+t)[1:]].location[x]=0
#             bpy.data.objects["Point."+str(1000+t)[1:]].location[y]=0
#             bpy.data.objects["Point."+str(1000+t)[1:]].location[z]=0
#             bpy.data.objects["Point."+str(1000+t)[1:]].keyframe_insert(data_path="location", frame=idx)
#             continue
# #    pose.close()


# def get_video_frames(file_url):
#     vidcap = cv2.VideoCapture(file_url)
#     success, image = vidcap.read()
#     global alldata_smoothed
#     # array of objects with class 'numpy.ndarray'
#     frames = []
#     while success:
# #        frames.append(image)
#         success, image = vidcap.read()
#         break
#     for i in range(int(alldata_smoothed.shape[0])):
#         frames.append(image)
# #
#     return frames

# def create_dots(name, amount):
#     #remove Collection
#     if bpy.data.collections.find(name) >= 0:
#         collection = bpy.data.collections.get(name)
#         #
#         for obj in collection.objects:
#             bpy.data.objects.remove(obj, do_unlink=True)
#         bpy.data.collections.remove(collection)
#     #cria os pontos nuima collection chamada Points
#     #=====================================================
#     collection = bpy.data.collections.new(name)
#     bpy.context.scene.collection.children.link(collection)
# #
#     layer_collection = bpy.context.view_layer.layer_collection.children[collection.name]
#     bpy.context.view_layer.active_layer_collection = layer_collection
# #
#     for point in range(amount):
#         bpy.ops.mesh.primitive_plane_add(enter_editmode=True, align='WORLD', location=(0, 0, 0), scale=(1, 1, 1))
#         bpy.ops.mesh.merge(type='CENTER')
#         bpy.ops.object.editmode_toggle()
#         bpy.context.active_object.name = name+'.'+str(1000+point)[1:]
#     #=====================================================
    
# #==============================
# #codes to size the bones
# #==============================

# def distance(point1, point2) -> float: 
#     #Calculate distance between two points in 3D.
# #    return math.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2 + (point2[2] - point1[2]) ** 2)
#     return math.sqrt((point2.location[0] - point1.location[0]) ** 2 + (point2.location[1] - point1.location[1]) ** 2 + (point2.location[2] - point1.location[2]) ** 2)


# def size_bone(point_name1, point_name2, bone):
#     p1 = bpy.data.objects[point_name1]
#     p2 = bpy.data.objects[point_name2]
#     #edit bones
#     if bpy.context.active_object.mode == 'EDIT':
#         bpy.context.object.data.edit_bones[bone].length= distance(p1,p2)
#     else:
#         bpy.ops.object.editmode_toggle()
#         bpy.context.object.data.edit_bones[bone].length= distance(p1,p2)
#     bpy.ops.object.editmode_toggle()


# ###### Change the path to you video on the line below.
# path = r'C:/Users/sutir/Downloads/blender/d.mp4'

# create_dots('Point',80)
# get_landmarks('Name', get_video_frames(path))


# #===========
# # selectign Scene Collection
# scene_collection = bpy.context.view_layer.layer_collection
# bpy.context.view_layer.active_layer_collection = scene_collection


# #===================================
# #creating bones
# #====================================
# bpy.ops.object.armature_add(enter_editmode=True, align='WORLD', location=(0, 0, 0), scale=(1, 1, 1)) #cria armature e primeiro bone
# #bpy.ops.object.editmode_toggle()
# #bpy.data.armatures['Armature'].edit_bones.active = bpy.context.object.data.edit_bones['Bone']


# obs = []
# for ob in bpy.context.scene.objects:
#     if ob.type == 'ARMATURE':
#         obs.append(ob)
# #obs


# bpy.ops.armature.select_all(action='DESELECT')
# #bpy.context.object.data.edit_bones['Bone'].select_tail=True
# obs[len(obs)-1].data.edit_bones['Bone'].select_tail=True
# #bpy.ops.armature.extrude_move()#Neck
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# #bpy.ops.armature.extrude_move()#Face
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})


# bpy.ops.armature.bone_primitive_add()#Arm_L
# #bpy.ops.armature.extrude_move()#Forearm_L 
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# #LeftHand
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})


# bpy.ops.armature.bone_primitive_add()#Arm_R
# #bpy.ops.armature.extrude_move()#Forearm_R
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# #RightHand
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})



# bpy.ops.armature.bone_primitive_add()#Thigh_L
# #bpy.ops.armature.extrude_move()#Leg_L
# #bpy.ops.armature.extrude_move()#Foot_L
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})

# bpy.ops.armature.bone_primitive_add()#Thigh_R
# #bpy.ops.armature.extrude_move()#Leg_R
# #bpy.ops.armature.extrude_move()#Foot_R
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})

# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})



# obs[len(obs)-1].data.edit_bones["Bone"].name = 'Spine'
# obs[len(obs)-1].data.edit_bones["Bone.001"].name = 'Neck'
# obs[len(obs)-1].data.edit_bones["Bone.002"].name = 'Face'
# obs[len(obs)-1].data.edit_bones["Bone.003"].name = 'Arm_L'
# obs[len(obs)-1].data.edit_bones["Bone.004"].name = 'Forearm_L'
# obs[len(obs)-1].data.edit_bones["Bone.005"].name = 'LeftHand'
# obs[len(obs)-1].data.edit_bones["Bone.006"].name = 'Arm_R'
# obs[len(obs)-1].data.edit_bones["Bone.007"].name = 'Forearm_R'
# obs[len(obs)-1].data.edit_bones["Bone.008"].name = 'RightHand'
# obs[len(obs)-1].data.edit_bones["Bone.009"].name = 'Thigh_L'
# obs[len(obs)-1].data.edit_bones["Bone.010"].name = 'Leg_L'
# obs[len(obs)-1].data.edit_bones["Bone.011"].name = 'Foot_L'
# obs[len(obs)-1].data.edit_bones["Bone.012"].name = 'Thigh_R'
# obs[len(obs)-1].data.edit_bones["Bone.013"].name = 'Leg_R'
# obs[len(obs)-1].data.edit_bones["Bone.014"].name = 'Foot_R'



# ###-------------------------------------creating Left hand finger bones----------------------------------

# ### Thumb_L
# bpy.ops.armature.select_all(action='DESELECT')
# obs[len(obs)-1].data.edit_bones['LeftHand'].select_tail=True
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})


# ### Index_L
# bpy.ops.armature.select_all(action='DESELECT')
# obs[len(obs)-1].data.edit_bones['LeftHand'].select_tail=True
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})



# ### Middle_L
# bpy.ops.armature.select_all(action='DESELECT')

# obs[len(obs)-1].data.edit_bones['LeftHand'].select_tail=True
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})




# ### Ring_L
# bpy.ops.armature.select_all(action='DESELECT')

# obs[len(obs)-1].data.edit_bones['LeftHand'].select_tail=True
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})


# ### Pinky_L
# bpy.ops.armature.select_all(action='DESELECT')

# obs[len(obs)-1].data.edit_bones['LeftHand'].select_tail=True
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})



# obs[len(obs)-1].data.edit_bones["LeftHand.001"].name = 'ThumbCMC_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.002"].name = 'ThumbMCP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.003"].name = 'ThumbIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.004"].name = 'ThumbTIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.005"].name = 'IndexMCP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.006"].name = 'IndexPIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.007"].name = 'IndexDIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.008"].name = 'IndexTIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.009"].name = 'MiddleMCP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.010"].name = 'MiddlePIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.011"].name = 'MiddleDIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.012"].name = 'MiddleTIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.013"].name = 'RingMCP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.014"].name = 'RingPIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.015"].name = 'RingDIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.016"].name = 'RingTIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.017"].name = 'PinkyMCP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.018"].name = 'PinkyPIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.019"].name = 'PinkyDIP_L'
# obs[len(obs)-1].data.edit_bones["LeftHand.020"].name = 'PinkyTIP_L'


# #-------------------------------------creating Right hand finger bones----------------------------------

# ### Thumb_R
# bpy.ops.armature.select_all(action='DESELECT')
# obs[len(obs)-1].data.edit_bones['RightHand'].select_tail=True
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})


# ### Index_R
# bpy.ops.armature.select_all(action='DESELECT')
# obs[len(obs)-1].data.edit_bones['RightHand'].select_tail=True
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})



# ### Middle_R
# bpy.ops.armature.select_all(action='DESELECT')

# obs[len(obs)-1].data.edit_bones['RightHand'].select_tail=True
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})




# ### Ring_R
# bpy.ops.armature.select_all(action='DESELECT')

# obs[len(obs)-1].data.edit_bones['RightHand'].select_tail=True
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})


# ### Pinky_R
# bpy.ops.armature.select_all(action='DESELECT')

# obs[len(obs)-1].data.edit_bones['RightHand'].select_tail=True
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})
# bpy.ops.armature.extrude_move(ARMATURE_OT_extrude={"forked":False}, TRANSFORM_OT_translate={"value":(0.1, 0.1, 0.1), "orient_type":'GLOBAL', "orient_matrix":((1, 0, 0), (0, 1, 0), (0, 0, 1)), "orient_matrix_type":'GLOBAL', "constraint_axis":(False, False, False), "mirror":True, "use_proportional_edit":False, "proportional_edit_falloff":'SMOOTH', "proportional_size":1, "use_proportional_connected":False, "use_proportional_projected":False, "snap":False, "snap_target":'CLOSEST', "snap_point":(0, 0, 0), "snap_align":False, "snap_normal":(0, 0, 0), "gpencil_strokes":False, "cursor_transform":False, "texture_space":False, "remove_on_cancel":False, "release_confirm":False, "use_accurate":False, "use_automerge_and_split":False})



# obs[len(obs)-1].data.edit_bones["RightHand.001"].name = 'ThumbCMC_R'
# obs[len(obs)-1].data.edit_bones["RightHand.002"].name = 'ThumbMCP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.003"].name = 'ThumbIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.004"].name = 'ThumbTIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.005"].name = 'IndexMCP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.006"].name = 'IndexPIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.007"].name = 'IndexDIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.008"].name = 'IndexTIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.009"].name = 'MiddleMCP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.010"].name = 'MiddlePIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.011"].name = 'MiddleDIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.012"].name = 'MiddleTIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.013"].name = 'RingMCP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.014"].name = 'RingPIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.015"].name = 'RingDIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.016"].name = 'RingTIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.017"].name = 'PinkyMCP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.018"].name = 'PinkyPIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.019"].name = 'PinkyDIP_R'
# obs[len(obs)-1].data.edit_bones["RightHand.020"].name = 'PinkyTIP_R'

# bpy.ops.object.editmode_toggle()





# #==========================================
# #selecting and making the armature Active
# #selecionando armature
# #==========================================

# armature = obs[len(obs)-1].name

# #bpy.data.objects[armature].select_set(True)
# obs[len(obs)-1].select_set(True)
# view_layer = bpy.context.view_layer
# #Armature_obj = bpy.context.scene.objects[armature]
# Armature_obj = obs[len(obs)-1]
# view_layer.objects.active = Armature_obj


# size_bone("Point.035", "Point.034", "Spine")
# size_bone("Point.034", "Point.033", "Neck")
# size_bone("Point.033", "Point.000", "Face")

# size_bone("Point.011", "Point.013", "Arm_L")
# size_bone("Point.013", "Point.015", "Forearm_L")


# size_bone("Point.036", "Point.078", "LeftHand")

# size_bone("Point.012", "Point.014", "Arm_R")
# size_bone("Point.014", "Point.016", "Forearm_R")
# size_bone("Point.057", "Point.079", "RightHand")

# size_bone("Point.023", "Point.025", "Thigh_L")
# size_bone("Point.025", "Point.027", "Leg_L")
# size_bone("Point.027", "Point.031", "Foot_L")

# size_bone("Point.024", "Point.026", "Thigh_R")
# size_bone("Point.026", "Point.028", "Leg_R")
# size_bone("Point.028", "Point.032", "Foot_R")

# #---------------------------------------------------bone size of Left hand fingers------------------------------------------

# #Thumb_L

# size_bone("Point.078", "Point.037", "ThumbCMC_L")
# size_bone("Point.037", "Point.038", "ThumbMCP_L")
# size_bone("Point.038", "Point.039", "ThumbIP_L")
# size_bone("Point.039", "Point.040", "ThumbTIP_L")

# #index_L
# size_bone("Point.078", "Point.041", "IndexMCP_L")
# size_bone("Point.041", "Point.042", "IndexPIP_L")
# size_bone("Point.042", "Point.043", "IndexDIP_L")
# size_bone("Point.043", "Point.044", "IndexTIP_L")

# #Middle_L
# size_bone("Point.078", "Point.045", "MiddleMCP_L")
# size_bone("Point.045", "Point.046", "MiddlePIP_L")
# size_bone("Point.046", "Point.047", "MiddleDIP_L")
# size_bone("Point.047", "Point.048", "MiddleTIP_L")

# #Ring_L
# size_bone("Point.078", "Point.049", "RingMCP_L")
# size_bone("Point.049", "Point.050", "RingPIP_L")
# size_bone("Point.050", "Point.051", "RingDIP_L")
# size_bone("Point.051", "Point.052", "RingTIP_L")

# #PInky_L
# size_bone("Point.053", "Point.054", "PinkyMCP_L")
# size_bone("Point.053", "Point.054", "PinkyPIP_L")
# size_bone("Point.054", "Point.055", "PinkyDIP_L")
# size_bone("Point.055", "Point.056", "PinkyTIP_L")

# ##---------------------------------------------------bone size of Right hand fingers------------------------------------------



# #Thumb_R

# size_bone("Point.078", "Point.037", "ThumbCMC_R")
# size_bone("Point.037", "Point.038", "ThumbMCP_R")
# size_bone("Point.038", "Point.039", "ThumbIP_R")
# size_bone("Point.039", "Point.040", "ThumbTIP_R")

# #index_R
# size_bone("Point.078", "Point.041", "IndexMCP_R")
# size_bone("Point.041", "Point.042", "IndexPIP_R")
# size_bone("Point.042", "Point.043", "IndexDIP_R")
# size_bone("Point.043", "Point.044", "IndexTIP_R")

# #Middle_R
# size_bone("Point.078", "Point.045", "MiddleMCP_R")
# size_bone("Point.045", "Point.046", "MiddlePIP_R")
# size_bone("Point.046", "Point.047", "MiddleDIP_R")
# size_bone("Point.047", "Point.048", "MiddleTIP_R")

# #Ring_R
# size_bone("Point.078", "Point.049", "RingMCP_R")
# size_bone("Point.049", "Point.050", "RingPIP_R")
# size_bone("Point.050", "Point.051", "RingDIP_R")
# size_bone("Point.051", "Point.052", "RingTIP_R")

# #PInky_R
# size_bone("Point.053", "Point.054", "PinkyMCP_R")
# size_bone("Point.053", "Point.054", "PinkyPIP_R")
# size_bone("Point.054", "Point.055", "PinkyDIP_R")
# size_bone("Point.055", "Point.056", "PinkyTIP_R")



# #comecando configuração  seguir movimentos pontos
# #colocando em pose mode
# bpy.ops.object.mode_set(mode='POSE')

# #bpy.data.objects[armature].pose.bones["Spine"]
# #bpy.data.objects[armature].pose.bones["Spine"].bone

# actual_bone = 'Spine'
# obs[len(obs)-1].data.bones.active = obs[len(obs)-1].pose.bones[actual_bone].bone
# obs[len(obs)-1].pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.035"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.034"]
# #=====
# actual_bone = 'Neck'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.033"]
# #=====
# actual_bone = 'Face'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.000"]
# #=====


# actual_bone = 'Arm_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.011"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.013"]
# #=====
# actual_bone = 'Forearm_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.013"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.036"]




# #=====

# actual_bone = 'Arm_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.012"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.014"]
# #=====
# actual_bone = 'Forearm_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.057"]


# actual_bone = 'Thigh_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.023"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.025"]
# #=====
# actual_bone = 'Leg_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.027"]
# #=====
# actual_bone = 'Foot_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.031"]
# #=====



# actual_bone = 'Thigh_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.024"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.026"]
# #=====
# actual_bone = 'Leg_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.028"]
# #=====
# actual_bone = 'Foot_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.032"]
# #=====

# #### #########################################Copy location and retarget of left hand and fingers################################################ 

# actual_bone = 'LeftHand'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.036"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.078"]

 
# #===== Thumb _L
# actual_bone = 'ThumbCMC_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.078"]


# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.037"]

# #===== 
# actual_bone = 'ThumbMCP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.037"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.038"]

# #=====
# actual_bone = 'ThumbIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.038"]


# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.039"]

# #===== 
# actual_bone = 'ThumbTIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.039"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.040"]


# #===== Index_L
# actual_bone = 'IndexMCP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.036"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.041"]
# #=====
# actual_bone = 'IndexPIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.041"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.042"]
# #=====
# actual_bone = 'IndexDIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True


# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.042"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.043"]
# #=====
# actual_bone = 'IndexTIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.043"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.044"]

# #===== Middle_L
# actual_bone = 'MiddleMCP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.036"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.045"]
# #=====
# actual_bone = 'MiddlePIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True


# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.045"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.046"]
# #=====
# actual_bone = 'MiddleDIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.046"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.047"]
# #=====
# actual_bone = 'MiddleTIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.047"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.048"]


# #===== Ring_R
# actual_bone = 'RingMCP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.036"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.049"]
# #=====
# actual_bone = 'RingPIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.049"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.050"]
# #=====
# actual_bone = 'RingDIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.050"]


# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.051"]
# #=====
# actual_bone = 'RingTIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.051"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.052"]


# #===== Pinky_R
# actual_bone = 'PinkyMCP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.036"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.053"]
# #=====
# actual_bone = 'PinkyPIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.053"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.054"]
# #=====
# actual_bone = 'PinkyDIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.054"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.055"]
# #=====
# actual_bone = 'PinkyTIP_L'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.055"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.056"]

# #=====

# #### #########################################Copy location and retarget of Right hand and fingers################################################ 

# actual_bone = 'RightHand'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.057"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.079"]


# #===== Thumb _L
# actual_bone = 'ThumbCMC_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.057"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.058"]

# #===== 
# actual_bone = 'ThumbMCP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.058"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.059"]

# #=====
# actual_bone = 'ThumbIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.059"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.060"]

# #===== 
# actual_bone = 'ThumbTIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.060"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.061"]


# #===== Index_L
# actual_bone = 'IndexMCP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.057"]

# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.062"]
# #=====
# actual_bone = 'IndexPIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.062"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.063"]
# #=====
# actual_bone = 'IndexDIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.063"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.064"]
# #=====
# actual_bone = 'IndexTIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.064"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.065"]

# #===== Middle_L
# actual_bone = 'MiddleMCP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.057"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.066"]
# #=====
# actual_bone = 'MiddlePIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.066"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.067"]
# #=====
# actual_bone = 'MiddleDIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.067"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.068"]
# #=====
# actual_bone = 'MiddleTIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.068"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.069"]


# #===== Ring_R
# actual_bone = 'RingMCP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.057"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.070"]
# #=====
# actual_bone = 'RingPIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.070"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.071"]
# #=====
# actual_bone = 'RingDIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.071"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.072"]
# #=====
# actual_bone = 'RingTIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.072"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.073"]


# #===== Pinky_R
# actual_bone = 'PinkyMCP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True

# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.057"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.074"]
# #=====
# actual_bone = 'PinkyPIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.074"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.075"]
# #=====
# actual_bone = 'PinkyDIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.075"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.076"]
# #=====
# actual_bone = 'PinkyTIP_R'
# bpy.context.object.data.bones.active = bpy.data.objects[armature].pose.bones[actual_bone].bone
# bpy.context.object.pose.bones[actual_bone].bone.select = True
# bpy.ops.pose.constraint_add(type='COPY_LOCATION')
# bpy.context.object.pose.bones[actual_bone].constraints[0].target = bpy.data.objects["Point.076"]
# bpy.ops.pose.constraint_add(type='DAMPED_TRACK')
# bpy.context.object.pose.bones[actual_bone].constraints[1].target = bpy.data.objects["Point.077"]
 
# #=======


# #bpy.data.objects['Armature'].pose.bones.items()

# frames =  int(alldata_smoothed.shape[0]) 

# bpy.context.scene.frame_end = frames

# bpy.ops.nla.bake(frame_start=1, frame_end=frames, visual_keying=True, clear_constraints=True, clear_parents=True, bake_types={'POSE'})
# bpy.ops.object.mode_set(mode='OBJECT')



# # ### apagar collection points criada
# collection = bpy.data.collections.get('Point')
# #
# for obj in collection.objects:
#     bpy.data.objects.remove(obj, do_unlink=True)
# bpy.data.collections.remove(collection)


